In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from itertools import combinations
import h5py
import tensorflow as tf
from pathlib import Path
import pandas as pd

from recovar import (
    RepresentationLearningSingleAutoencoder, 
    RepresentationLearningDenoisingSingleAutoencoder, 
    RepresentationLearningMultipleAutoencoder
)
from config import BATCH_SIZE, WINDOW_SIZE
from direct_trainer import HDF5Generator

2025-07-06 15:48:41.186919: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-06 15:48:41.205379: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-06 15:48:41.205396: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-06 15:48:41.205406: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-06 15:48:41.208978: I tensorflow/core/platform/cpu_feature_g

In [5]:
# -------------------------------
# Configuration and Setup
# -------------------------------

# Choose the representation learning model class
REPRESENTATION_LEARNING_MODEL_CLASS = RepresentationLearningMultipleAutoencoder

# Path to the trained model weights
MODEL_WEIGHTS_PATH = "/home/ege/recovar/reproducibility/checkpoints/stead_full/FULL_STEAD_SUBSAMPLED_100_train_epoch_10.h5"

# Path to the test dataset
TEST_DATASET_PATH = "/home/ege/recovar/reproducibility/preprocessed_data/new_stead/FULL_STEAD_SUBSAMPLED_100_test.hdf5"

# Number of samples to visualize
NUM_SAMPLES = 50

# Batch size for processing
BATCH_SIZE = 256

In [ ]:
print("Loading metadata for crop offset filtering...")
with h5py.File(TEST_DATASET_PATH, 'r') as f:
    if 'metadata' in f:
        metadata_json = f['metadata'][()].decode('utf-8')
        metadata = pd.read_json(metadata_json)
        
        crop_filter = CropOffsetFilter()
        filtered_metadata = crop_filter.apply(metadata)
        
        #Get indices that passed the filter
        filtered_indices = filtered_metadata.index.values
        
        print(f"Crop offset filter: {len(filtered_indices)}/{len(metadata)} samples passed")
    else:
        print("Warning: No metadata found in HDF5 file. Skipping crop offset filter.")
        apply_crop_offset_filter = False

In [6]:
# -------------------------------
# Define Utility Functions
# -------------------------------

def compute_covariance(data_arrays):
    """
    Computes autocovariance or cross-covariance between signals.
    
    Args:
        data_arrays (np.ndarray): Variable number of 2D arrays, each with shape (timesteps, channels)
    
    Returns:
        lags (np.ndarray): Lag values
        avg_cov (np.ndarray): Averaged covariance
    """
    if (len(np.shape(data_arrays)) == 2):
        data_arrays = np.expand_dims(data_arrays, axis=0)
    
    num_signals = len(data_arrays)    
    num_timesteps, num_channels = data_arrays[0].shape

    covariances = []
    lags = np.arange(-num_timesteps + 1, num_timesteps)

    if num_signals == 1:
        # Autocovariance
        data = data_arrays[0]
        for c in range(num_channels):
            channel_data = data[:, c]
            channel_data = channel_data - np.mean(channel_data)  # Zero-mean
            cov = np.correlate(channel_data, channel_data, mode='full')
            covariances.append(cov)
    else:
        # Cross-covariance between all possible pairs
        pairs = list(combinations(range(num_signals), 2))
        for idx1, idx2 in pairs:
            data1 = data_arrays[idx1]
            data2 = data_arrays[idx2]
            for c in range(num_channels):
                channel_data1 = data1[:, c]
                channel_data2 = data2[:, c]
                channel_data1 = channel_data1 - np.mean(channel_data1)  # Zero-mean
                channel_data2 = channel_data2 - np.mean(channel_data2)  # Zero-mean
                cov = np.correlate(channel_data1, channel_data2, mode='full')
                covariances.append(cov)

    covariances = np.array(covariances)
    avg_cov = np.mean(covariances, axis=0)
    return lags, avg_cov

def plot_waveform_channel(ax, timesteps, waveform, channel_idx, ylim_min=None, ylim_max=None, color='blue', show_xticks=True):
    """
    Plots a single waveform channel on the given axes.
    """
    channels = ['E', 'N', 'Z']
    ax.plot(timesteps, waveform, color=color, linewidth=1)
    if channel_idx == 0:
        ax.set_title("Waveform", fontsize=14, pad=5, fontweight='bold')
        
    if ylim_min != None and ylim_max != None:
        ax.set_ylim(ymin=ylim_min, ymax=ylim_max)
    
    ax.tick_params(axis='y', labelsize=10)
    if show_xticks:
        ax.set_xlabel('Timesteps', fontsize=12)
        ax.tick_params(axis='x', labelsize=10)
    else:
        ax.set_xlabel('')
        ax.set_xticklabels([])
        ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    
    ax.grid(True)

def plot_heatmap(ax, heatmap, vmin=None, vmax=None, title=None):
    """
    Plots the heatmap on the given axes.
    """
    if vmin != None and vmax != None:
        cax = ax.imshow(heatmap, aspect='auto', cmap='magma', origin='lower', vmin=vmin, vmax=vmax)
    else:
        cax = ax.imshow(heatmap, aspect='auto', cmap='magma', origin='lower')
        
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.set_xlabel('Timesteps', fontsize=12)
    ax.set_ylabel('Channels', fontsize=12)
    plt.colorbar(cax, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)

def plot_autocovariance(ax, lags, autocov, ylim_min=None, ylim_max=None, title=None, ylabel='Autocovariance'):
    """
    Plots the autocovariance function on the given axes.
    """
    if ylim_min != None and ylim_max != None:
        ax.set_ylim(ymin=ylim_min, ymax=ylim_max)
        
    ax.plot(lags, autocov)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.set_xlabel('Lag', fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)

def load_model():
    """
    Load the model and weights.
    """
    # Initialize the representation learning model
    model = REPRESENTATION_LEARNING_MODEL_CLASS()
    model.compile(optimizer=tf.keras.optimizers.Adam())

    # Build the model (lazy building)
    dummy_input = tf.zeros((1, 3000, 3))
    model(dummy_input)

    # Load the pre-trained weights
    if Path(MODEL_WEIGHTS_PATH).exists():
        model.load_weights(MODEL_WEIGHTS_PATH)
        print(f"Loaded weights from: {MODEL_WEIGHTS_PATH}")
    else:
        raise FileNotFoundError(f"Model weights not found at: {MODEL_WEIGHTS_PATH}")
    
    return model

def load_sample_data_direct(test_dataset_path, num_samples):
    """
    Load sample data using the direct method (from HDF5 file).
    
    Returns:
        waveforms (np.ndarray): Shape (num_samples, 3000, 3)
        labels (np.ndarray): Shape (num_samples)
        metadata (pd.DataFrame or None): Metadata if available
    """
    # Use HDF5Generator to load data
    test_gen = HDF5Generator(test_dataset_path, batch_size=BATCH_SIZE, shuffle=False)
    
    # Load metadata if available
    metadata = None
    with h5py.File(test_dataset_path, 'r') as f:
        if 'metadata' in f:
            metadata_json = f['metadata'][()].decode('utf-8')
            metadata = pd.read_json(metadata_json)
            print(f"Loaded metadata with {len(metadata)} samples")
    
    # Calculate number of batches needed
    num_batches = min((num_samples + BATCH_SIZE - 1) // BATCH_SIZE, len(test_gen))
    
    X = []
    Y = []
    
    # Load data batch by batch
    for i in range(num_batches):
        x_batch, y_batch = test_gen[i]
        X.append(x_batch)
        Y.append(y_batch)
        
        # Break if we have enough samples
        if sum(len(x) for x in X) >= num_samples:
            break
    
    # Concatenate and trim to exact number of samples
    X = np.concatenate(X, axis=0)[:num_samples]
    Y = np.concatenate(Y, axis=0)[:num_samples]
    
    print(f"Loaded {len(X)} samples")
    print(f"Earthquake samples: {np.sum(Y == 1)}")
    print(f"Noise samples: {np.sum(Y == 0)}")
    
    return X, Y, metadata

In [7]:
# Load the model and data
model = load_model()
waveforms, labels, metadata = load_sample_data_direct(TEST_DATASET_PATH, NUM_SAMPLES)

# Get latent representations
if REPRESENTATION_LEARNING_MODEL_CLASS == RepresentationLearningMultipleAutoencoder:
    model_out = model(waveforms)
    feature_maps = list(model_out)[0:5]  # Get the 5 feature maps
    feature_maps = np.array(feature_maps)
else:
    model_out = model(waveforms)
    feature_maps = list(model_out)[0:1]  # Get single feature map
    feature_maps = np.array(feature_maps)
    
# Transpose to have batch dimension first
feature_maps = np.transpose(feature_maps, axes=[1, 0, 2, 3])

print(f"Feature maps shape: {feature_maps.shape}")

Loaded weights from: /home/ege/recovar/reproducibility/checkpoints/stead_full/FULL_STEAD_SUBSAMPLED_100_train_epoch_10.h5
Loaded metadata with 253132 samples
Loaded 50 samples
Earthquake samples: 42
Noise samples: 8
Feature maps shape: (50, 5, 94, 64)


In [ ]:
# Visualize latent representations for earthquake vs noise samples
WAVEFORM_COLORS = ['blue', 'green', 'red']  # For E, N, Z channels

# Separate earthquake and noise indices
earthquake_indices = [i for i, label in enumerate(labels) if label > 0.5]
noise_indices = [i for i, label in enumerate(labels) if label <= 0.5]

# Ensure we have both earthquake and noise samples
NUM_PLOTS = min(len(earthquake_indices), len(noise_indices), 10)  # Limit to 10 plots

if NUM_PLOTS == 0:
    print("Not enough earthquake or noise samples to create comparison plots.")
else:
    print(f"Creating {NUM_PLOTS} comparison plots...")

for plot_idx in range(NUM_PLOTS):
    eq_idx = earthquake_indices[plot_idx]
    noise_idx = noise_indices[plot_idx]
    
    # Extract earthquake data
    eq_waveform = waveforms[eq_idx]
    eq_feature_map = feature_maps[eq_idx]
    lags_waveform_eq, autocov_waveform_eq = compute_covariance(eq_waveform)
    lags_heatmap_eq, autocov_heatmap_eq = compute_covariance(eq_feature_map)
    
    # Extract noise data
    noise_waveform = waveforms[noise_idx]
    noise_feature_map = feature_maps[noise_idx]
    lags_waveform_noise, autocov_waveform_noise = compute_covariance(noise_waveform)
    lags_heatmap_noise, autocov_heatmap_noise = compute_covariance(noise_feature_map)
    
    # Create figure
    fig = plt.figure(figsize=(20, 10))
    main_gs = GridSpec(1, 2, figure=fig, wspace=0.3)
    
    # --- Earthquake Column ---
    eq_gs = main_gs[0, 0].subgridspec(2, 2, wspace=0.3, hspace=0.4)
    
    # Top-Left: Waveform Channels
    eq_waveform_gs = eq_gs[0, 0].subgridspec(eq_waveform.shape[1], 1, hspace=0.3)
    timesteps_eq = np.arange(eq_waveform.shape[0])
    
    for channel in range(eq_waveform.shape[1]):
        ax = fig.add_subplot(eq_waveform_gs[channel, 0])
        show_xticks = (channel == eq_waveform.shape[1] - 1)
        plot_waveform_channel(ax, timesteps_eq, eq_waveform[:, channel], channel, 
                              color=WAVEFORM_COLORS[channel % len(WAVEFORM_COLORS)], 
                              show_xticks=show_xticks)
    
    # Top-Right: Heatmap
    ax_heatmap_eq = fig.add_subplot(eq_gs[0, 1])
    plot_heatmap(ax_heatmap_eq, eq_feature_map[0].T, title="Latent Representation")
    
    # Bottom-Left: Autocovariance of Waveform
    ax_autocov_waveform_eq = fig.add_subplot(eq_gs[1, 0])
    plot_autocovariance(ax_autocov_waveform_eq, lags_waveform_eq, autocov_waveform_eq, 
                        title='Waveform\nAutocovariance function')
    
    if REPRESENTATION_LEARNING_MODEL_CLASS == RepresentationLearningMultipleAutoencoder:
        latent_covariance_title = 'Latent Representation\nCross-covariance function'
        latent_covariance_ylabel = 'Mean Cross-covariance'
    else:
        latent_covariance_title = 'Latent Representation\nAuto-covariance function'
        latent_covariance_ylabel = 'Auto-covariance'
        
    # Bottom-Right: Autocovariance of Heatmap
    ax_autocov_heatmap_eq = fig.add_subplot(eq_gs[1, 1])
    plot_autocovariance(ax_autocov_heatmap_eq, lags_heatmap_eq, autocov_heatmap_eq, 
                        title=latent_covariance_title,
                        ylabel=latent_covariance_ylabel)
    
    # --- Noise Column ---
    # Use same scale limits for noise plots
    feature_map_max = np.max(eq_feature_map[0], axis=(0, 1))
    feature_map_min = np.min(eq_feature_map[0], axis=(0, 1))
    
    autocov_heatmap_max = np.max(autocov_heatmap_eq, axis=(0))
    autocov_heatmap_min = np.min(autocov_heatmap_eq, axis=(0))
    
    noise_gs = main_gs[0, 1].subgridspec(2, 2, wspace=0.3, hspace=0.4)
    
    # Top-Left: Waveform Channels
    noise_waveform_gs = noise_gs[0, 0].subgridspec(noise_waveform.shape[1], 1, hspace=0.3)
    timesteps_noise = np.arange(noise_waveform.shape[0])
    
    for channel in range(noise_waveform.shape[1]):
        ax = fig.add_subplot(noise_waveform_gs[channel, 0])
        show_xticks = (channel == noise_waveform.shape[1] - 1)
        plot_waveform_channel(ax, timesteps_noise, noise_waveform[:, channel], channel, 
                              color=WAVEFORM_COLORS[channel % len(WAVEFORM_COLORS)], 
                              show_xticks=show_xticks)
    
    # Top-Right: Heatmap
    ax_heatmap_noise = fig.add_subplot(noise_gs[0, 1])
    plot_heatmap(ax_heatmap_noise, noise_feature_map[0].T, feature_map_min, feature_map_max, "Latent Representation")
    
    # Bottom-Left: Autocovariance of Waveform
    ax_autocov_waveform_noise = fig.add_subplot(noise_gs[1, 0])
    plot_autocovariance(ax_autocov_waveform_noise, 
                        lags_waveform_noise, 
                        autocov_waveform_noise, 
                        title='Waveform\nAutocovariance function')
    
    # Bottom-Right: Autocovariance of Heatmap
    ax_autocov_heatmap_noise = fig.add_subplot(noise_gs[1, 1])
    plot_autocovariance(ax_autocov_heatmap_noise, 
                        lags_heatmap_noise, 
                        autocov_heatmap_noise,
                        autocov_heatmap_min,
                        autocov_heatmap_max,
                        latent_covariance_title,
                        latent_covariance_ylabel)
    
    # Add column titles
    fig.text(0.30, 0.935, 'Earthquake sample', ha='center', va='center', fontsize=20, fontweight='bold')
    fig.text(0.725, 0.935, 'Noise sample', ha='center', va='center', fontsize=20, fontweight='bold')
    
    # Adjust layout and save
    plt.tight_layout(rect=[0, 0.03, 0.03, 0.75])
    
    # Create output directory if it doesn't exist
    output_dir = Path("latent_visualizations_direct")
    output_dir.mkdir(exist_ok=True)
    
    output_path = output_dir / f"latent_plot_pair_{plot_idx + 1}.png"
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close(fig)
    
    print(f"Saved plot {plot_idx + 1} to {output_path}")

print("\nLatent space visualization complete!")

In [ ]:
# Additional analysis: Plot average latent representations for earthquake vs noise
if len(earthquake_indices) > 0 and len(noise_indices) > 0:
    # Get average feature maps
    eq_feature_maps = feature_maps[earthquake_indices[:min(50, len(earthquake_indices))]]
    noise_feature_maps = feature_maps[noise_indices[:min(50, len(noise_indices))]]
    
    avg_eq_feature = np.mean(eq_feature_maps[:, 0, :, :], axis=0)
    avg_noise_feature = np.mean(noise_feature_maps[:, 0, :, :], axis=0)
    
    # Plot comparison
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Average earthquake latent representation
    im1 = axes[0].imshow(avg_eq_feature.T, aspect='auto', cmap='magma', origin='lower')
    axes[0].set_title('Average Earthquake\nLatent Representation', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Timesteps')
    axes[0].set_ylabel('Channels')
    plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)
    
    # Average noise latent representation
    im2 = axes[1].imshow(avg_noise_feature.T, aspect='auto', cmap='magma', origin='lower')
    axes[1].set_title('Average Noise\nLatent Representation', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Timesteps')
    axes[1].set_ylabel('Channels')
    plt.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04)
    
    # Difference
    diff = avg_eq_feature - avg_noise_feature
    im3 = axes[2].imshow(diff.T, aspect='auto', cmap='RdBu_r', origin='lower')
    axes[2].set_title('Difference\n(Earthquake - Noise)', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('Timesteps')
    axes[2].set_ylabel('Channels')
    plt.colorbar(im3, ax=axes[2], fraction=0.046, pad=0.04)
    
    plt.tight_layout()
    
    output_path = Path("latent_visualizations_direct") / "average_latent_comparison.png"
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nSaved average comparison to {output_path}")

In [ ]:
# If using multiple autoencoders, visualize differences between them
if REPRESENTATION_LEARNING_MODEL_CLASS == RepresentationLearningMultipleAutoencoder and len(earthquake_indices) > 0:
    sample_idx = earthquake_indices[0]  # Take first earthquake sample
    sample_features = feature_maps[sample_idx]  # Shape: (5, 94, 64)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    # Plot each autoencoder's representation
    for i in range(min(5, sample_features.shape[0])):
        im = axes[i].imshow(sample_features[i].T, aspect='auto', cmap='magma', origin='lower')
        axes[i].set_title(f'Autoencoder {i+1}\nLatent Representation', fontsize=12, fontweight='bold')
        axes[i].set_xlabel('Timesteps')
        axes[i].set_ylabel('Channels')
        plt.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)
    
    # Hide unused subplot
    axes[5].axis('off')
    
    plt.suptitle('Multiple Autoencoder Representations for Single Earthquake Sample', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
